In [197]:
import brainlit
from brainlit.utils.session import NeuroglancerSession
import matplotlib.pyplot as plt
import numpy as np
import napari
from scipy import ndimage


url = "s3://open-neurodata/brainlit/brain1"
ngl = NeuroglancerSession(url, mip=2)
seg_id = 7
v_id = 1
img, bounds, voxel = ngl.pull_chunk(seg_id=seg_id, v_id=v_id)
napari.view_image(img)

Downloading:   0%|          | 0/1 [00:00<?, ?it/s]


### threshold top 5%

In [231]:
H = np.sort(np.reshape(img,[1,-1]))
thres = H[0,-round((H.shape[1])*0.05)]
lx,ly,lz = img.shape
imgTop5 = np.zeros([lx,ly,lz])
for m in range(lx):
    for n in range(ly):
        for p in range(lz):
            if img[m,n,p] < thres:
                imgTop5[m,n,p] = 0
            else:
                imgTop5[m,n,p] = img[m,n,p]

viewer = napari.view_image(img, name='Original Image', colormap='red')
viewer.add_labels(imgTop5, name='Gray Threshold at 5%', num_colors=2, opacity=0.3)
        

<Labels layer 'Gray Threshold at 5%' at 0x24f6469b748>

### Binarize at top 0.1%

In [234]:
H = np.sort(np.reshape(img,[1,-1]))
thres = H[0,-round((H.shape[1])*0.001)]
lx,ly,lz = img.shape
imgBW = np.zeros([lx,ly,lz])
for m in range(lx):
    for n in range(ly):
        for p in range(lz):
            if img[m,n,p] < thres:
                imgBW[m,n,p] = 0
            else:
                imgBW[m,n,p] = 1

viewer = napari.view_image(img, name='Original Image', colormap='red')
viewer.add_labels(imgBW, name='Binary Threshold at 5%', num_colors=2, opacity=0.5)

<Labels layer 'Binary Threshold at 5%' at 0x24f5e1ab148>

### gray erosion

In [235]:
Gero = ndimage.grey_erosion(imgTop5, size=2)

viewer = napari.view_image(img, name='Original Image', colormap='red')
viewer.add_labels(Gero, name='Gray Erosion', num_colors=2, opacity=0.5)

<Labels layer 'Gray Erosion' at 0x24f615609c8>

### binary erosion

In [236]:
ero5 = ndimage.binary_erosion(imgTop5, iterations=1)

viewer = napari.view_image(img, name='Original Image', colormap='red')
viewer.add_labels(ero5, name='Gray Erosion', num_colors=2, opacity=0.5)

<Labels layer 'Gray Erosion' at 0x24f6172d408>